In [32]:
# pip install qdrant-client sentence-transformers
# docker run -p 6333:6333 qdrant/qdrant

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

texts = [
    "Vector databases are designed for similarity search.",
    "Qdrant is an open-source vector database.",
    "Machine learning models produce embeddings.",
    "We can build semantic search using embeddings."
]

In [5]:
embeddings = model.encode(texts)
print(embeddings.shape)  # (4, 384)

(4, 384)


In [7]:
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest

# Connect to local Qdrant
client = QdrantClient(host="localhost", port=6333)

In [9]:
# Create a collection
client.recreate_collection(
    collection_name="docs",
    vectors_config=rest.VectorParams(size=384, distance=rest.Distance.COSINE),
)

/var/folders/bw/9yp1yw051tb99202n73fdz8c0000gp/T/ipykernel_8530/500860329.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [19]:
client.upsert(
    collection_name="docs",
    points=[
        rest.PointStruct(id=i, vector=embeddings[i].tolist(), payload={"text": texts[i]})
        for i in range(len(texts))
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [31]:
query = "How do I build semantic search?"
query_emb = model.encode(query).tolist()
results = client.search(
    collection_name="docs",
    query_vector=query_emb,
    limit=2
)

for r in results:
    print(r.payload["text"], " (score:", r.score, ")")


We can build semantic search using embeddings.  (score: 0.7714833 )
Vector databases are designed for similarity search.  (score: 0.42973614 )


/var/folders/bw/9yp1yw051tb99202n73fdz8c0000gp/T/ipykernel_8530/3403734425.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
